# Contextual Synonym Analysis (NLTK Version 2)

This notebook identifies interchangeable words using NLTK WordNet and edit distance (Version 2).

In [1]:
import json
from nltk.corpus import wordnet as wn
from collections import defaultdict

THRESHOLD = 1
def extract_vocab(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        return []
        
    vocab_keys = data.get('model', {}).get('vocab', {}).keys()
    
    # Fast set comprehension for filtering
    return {w for w in vocab_keys 
            if len(w) > 3 and w.isalpha() and not w.startswith("##")}

def find_contextual_synonyms(vocab_set):
    print(f"Building indices for {len(vocab_set)} words...")
    
    # Build Lookup Tables (The "Inverted Index")
    word_to_synsets = {}
    
    synset_to_words = defaultdict(list)
    
    for word in vocab_set:
        synsets = set(syn.name() for syn in wn.synsets(word))
        
        if synsets:
            word_to_synsets[word] = synsets
            for sid in synsets:
                synset_to_words[sid].append(word)
                
    print("Calculating overlaps...")
    
    results = []
    processed_pairs = set()

    # Iterate through words and find their "Contextual Twins"
    for word_a, contexts_a in word_to_synsets.items():
        
        candidates = set()
        for sid in contexts_a:
            candidates.update(synset_to_words[sid])

        
        for word_b in candidates:
            if word_a == word_b:
                continue
            # Sort pair to avoid duplicates: (A, B) and (B, A)
            pair_key = tuple(sorted((word_a, word_b)))
            if pair_key in processed_pairs:
                continue
            
            # Calculate Similarity (Jaccard Index)
            contexts_b = word_to_synsets[word_b]
            
            intersection = len(contexts_a.intersection(contexts_b))
            union = len(contexts_a)
            score = intersection / union
            
            if score >= THRESHOLD :
                # Adding pair key only if score > Threshold allows alg to look both ways if one word has less definitions
                processed_pairs.add(pair_key)
                results.append({
                    "word_a": word_a,
                    "word_b": word_b,
                    "score": round(score, 2),
                    "shared_contexts": intersection,
                    "total_contexts": union
                })

    return results


vocab = extract_vocab("./bert_tokenizer_uncased/tokenizer.json")
#vocab = None

if not vocab:
    vocab = {"looked", "expected", "movie"}

# Find Pairs
interchangeable_pairs = find_contextual_synonyms(vocab)
interchangeable_pairs.sort(key=lambda x: x['score'], reverse=True)

print(f"\nFound {len(interchangeable_pairs)} highly interchangeable pairs.\n")
print(f"{'Word A':<15} | {'Replacement':<15} | {'Score':<5} | {'Shared Meanings'}")
print("-" * 60)

for p in interchangeable_pairs[:4000]:
    print(f"{p['word_a']:<15} | {p['word_b']:<15} | {p['score']:<5} | {p['shared_contexts']}/{p['total_contexts']}")

Building indices for 17582 words...
Calculating overlaps...

Found 6386 highly interchangeable pairs.

Word A          | Replacement     | Score | Shared Meanings
------------------------------------------------------------
draws           | draw            | 1.0   | 45/45
encountering    | encountered     | 1.0   | 5/5
encountering    | encounters      | 1.0   | 5/5
encountering    | encounter       | 1.0   | 5/5
competitor      | contender       | 1.0   | 1/1
competitor      | rival           | 1.0   | 1/1
competitor      | rivals          | 1.0   | 1/1
competitor      | contenders      | 1.0   | 1/1
competitor      | competitions    | 1.0   | 1/1
competitor      | competitors     | 1.0   | 1/1
competitor      | challenger      | 1.0   | 1/1
competitor      | competition     | 1.0   | 1/1
tanks           | tank            | 1.0   | 8/8
reproduced      | reproduce       | 1.0   | 4/4
recalls         | recall          | 1.0   | 12/12
recognised      | recognized      | 1.0   | 10/10
di

In [ ]:
def generate_replacement_map(pairs_list):
    queue = sorted(pairs_list, key=lambda x: x['score'], reverse=True)
    replacement_map = {}
    
    print(f"Starting reduction on {len(queue)} pairs...")
    
    while queue:
        # Pop the best remaining pair
        current_pair = queue.pop(0)
        
        loser = current_pair['word_a']
        winner = current_pair['word_b']
        
        # Add to Dictionary
        replacement_map[loser] = winner
        
        # Back-Propagate (Fix Chains)

        for key, val in replacement_map.items():
            if val == loser:
                replacement_map[key] = winner

        # Forward-Propagate (Update the waiting list)

        #I dont want to replace the remaing queue word 'next_word' a with the loser because loser is a subset of winner as well as next_word. but winner may not be a subset of next_word
        #If word_a == loser --> remove from the data set
        #However is next word is a subset of loser, that makes it a subset of winner
        new_queue = []
        for p in queue:
            # Swap 'loser' to 'winner' in all future pairs
            if p['word_a'] == loser:
                continue #do not add to queue
            if p['word_b'] == loser:
                p['word_b'] = winner
            
            # Only keep the pair if it's still useful
            if p['word_a'] != p['word_b']:
                new_queue.append(p)
                
        queue = new_queue
    return replacement_map


vocab_map = generate_replacement_map(interchangeable_pairs)


print(f"\nReduction Complete. Generated {len(vocab_map)} mappings.")
print(f"{'Original':<15} -> {'Replacement'}")
print("-" * 30)


for original, replacement in list(vocab_map.items())[:20]:
    print(f"{original:<15} -> {replacement}")

# Verify Chains are Flattened

chained_keys = [k for k, v in vocab_map.items() if v in vocab_map]
if len(chained_keys) == 0:
    print("\nSuccess: No daisy-chains found. All words map directly to their final target.")
else:
    print(f"\nWarning: {len(chained_keys)} chains remain.")

removed_words = list(vocab_map.keys())
mapping = vocab_map

UnboundLocalError: cannot access local variable 'queue' where it is not associated with a value

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./bert_tokenizer_uncased")
model_state = tokenizer.get_vocab()
for word in removed_words:
    model_state.pop(word, None)

print(model_state)
vocab_list = [token for token, idx in sorted(model_state.items(), key=lambda x: x[1])]
print(vocab_list)
print(len(vocab_list), "words in new vocab")

import json
with open("./wordnet_v2/filtered_tokenizer_vocab_wordnet_v2.json", "w", encoding="utf-8") as f:
    json.dump(vocab_list, f, ensure_ascii=False, indent=2)


with open("./wordnet_v2/removed_words_mapping_wordnet_v2.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

{'jared': 27177, '御': 2244, 'idaho': 20747, 'workshops': 24111, 'adj': 8956, 'horror': 11465, 'ក': 1006, 'density': 13636, 'basidia': 29347, '##hr': 10591, '寬': 2069, '##ulent': 19652, 'augustus': 17402, 'leopard': 21564, 'health': 7619, '代': 1527, 'humili': 25148, 'corresponding': 15606, '##sson': 26866, '先': 1625, 'saigon': 22660, 'beds': 16073, 'tna': 13606, 'ǝ': 135, '##ъ': 4452, '##ⲩ': 4863, '服': 2485, '##stones': 21767, 'judges': 14051, 'sixteenth': 20559, '##burg': 7920, 'gateway': 21474, '寞': 2064, '雉': 3857, 'aerial': 13030, 'barric': 21230, 'rotated': 28591, 'insur': 11437, 'intelligent': 15916, 'planted': 18851, 'mile': 8950, 'junct': 26640, 'svalbard': 27429, 'speculated': 16368, '付': 1525, 'principles': 13672, 'collectors': 22101, 'transcription': 26820, '誠': 3525, 'sms': 18199, 'motte': 25959, '##sp': 5866, 'seymour': 19530, '##anged': 20955, 'ceas': 11173, 'prefecture': 26474, '哈': 1841, 'voluntarily': 27008, '愚': 2299, '##rama': 25671, 'beec': 29105, 'bohem': 19095, 'vi